In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import plot_importance
import xgboost
import eli5 
from eli5.sklearn import PermutationImportance 
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
filepath_base = 'D:/Programming/CS/DGS_Analyze/DGS_Analyze_230912/bin/Debug/Result/'
modelname = ['EBR32094107-BOT' , 'EBR36323401-BOT' , 'EBR39988201-BOT' , 'EBR41041401-BOT']
filelist = []
for j in modelname:
    filepath = filepath_base + j +'/'
    filealllist =  os.listdir(filepath)
    
    for i in filealllist:
        if i.__contains__('PrjResult.csv'):
            pdsort = pd.read_csv(filepath + i , sep=',' , dtype = 'unicode')
            partcnt = pdsort['X'].count()

            if j == 'EBR32094107-BOT':
                if partcnt != 664:
                    continue
                if not (i[:3] == '020' or i[:3] == '030'):
                    continue
                if not (i[3:5] == '20' or i[3:5] == '30' or i[3:5] =='40'):
                    continue
                if i[10] != 'C':
                    continue

            elif j == 'EBR39988201-BOT':
                if partcnt != 603:
                    continue
                if not (i[:3] == '020' or i[:3] == '030'):
                    continue
                if not (i[3:5] == '20' or i[3:5] == '30' or i[3:5] =='40'):
                    continue
                if i[10] != 'C':
                    continue
            elif j == 'EBR36323401-BOT':
                if partcnt != 644:
                    continue
                if not (i[:3] == '020' or i[:3] == '025' or i[:3] == '030'):
                    continue
                if not (i[3:5] == '20' or i[3:5] == '30'):
                    continue
                if i[10] != 'C':
                    continue
            elif j == 'EBR41041401-BOT':
                if partcnt != 597:
                    continue
                if not (i[:3] == '020'):
                    continue
                if not (i[3:5] == '20' or i[3:5] == '30'):
                    continue
                if i[10] != 'C':
                    continue
            else:
                continue

            filelist.append(filepath+i)

In [3]:
len(filelist)

2075

In [20]:
PartVector_pd = pd.read_csv('PartVector_230927.csv' , sep=',' , dtype = 'unicode')
def PartVector_V(s):
    k = PartVector_pd[PartVector_pd['NAME'] == s]
    N = k.iloc[0][['V1' , 'V2' , 'V3']]
    return N


In [56]:
#################################### 단일 수량으로 TEST ##############################
#최대 부품 수량 정하기
max_partcnt = 1000

#prj_result 파일 읽어와서 부품의 벡터값 추가
path = filelist[0]
#prj_result 파일 읽어와서 부품의 벡터값 추가
read_pd = pd.read_csv(path , sep=',' , dtype = 'unicode')
read_pd[['V1' , 'V2' , 'V3']] = read_pd['NAME_Part'].apply(PartVector_V)

# 학습에 사용할 컬럼 추출
all_col = ['V1' , 'V2' , 'V3' , 'X' ,	'Y' , 'NOZZLE_A' , 'NOZZLE_B' , 'NOZZLE_C' , 'SupplyKind_num' , 'HEAD']
need_col = ['V1' , 'V2' , 'V3' , 'X' ,	'Y' , 'NOZZLE_A' , 'NOZZLE_B' , 'NOZZLE_C' , 'SupplyKind_num']
target_col = ['HEAD']
all_pd = read_pd[all_col]

# Option값 가져와서 넘파이로 변환
prj_opt = read_pd.loc[1][:14]
opt_np = prj_opt.to_numpy()
opt_np = opt_np.reshape(1,-1)
# print(opt_np)

# X,Y 좌표 크기로 정렬
all_pd['X'] = pd.to_numeric(all_pd['X'])
all_pd['Y'] = pd.to_numeric(all_pd['Y'])
all_pd = all_pd.sort_values(by = 'X')
all_pd = all_pd.sort_values(by = 'Y')
all_pd = all_pd.reset_index(drop=True)
prj_pd = all_pd[need_col]

# 부품데이터 넘파이로 변환
prj_np = prj_pd.to_numpy()
prj_np = prj_np.reshape(1, -1)
# print(prj_np)

# Option과 부품데이터 합치기
opt_need_np = np.concatenate([opt_np , prj_np] , axis=1)
print(opt_np.shape , prj_np.shape )
print(opt_need_np.shape , opt_need_np)

# Max 부품 수량에서 부족수량 채우기
spare_cnt = max_partcnt * len(need_col) - prj_np.shape[1]
spare_zero = np.zeros(spare_cnt)
spare_zero = spare_zero.reshape(1,-1)
print(spare_zero.shape)

# Option + 부품데이터 + 부족수량 합치기
opt_need_spare_np = np.concatenate([opt_need_np , spare_zero] , axis=1)
print(opt_need_spare_np.shape)

# Taget 데이터 넘파이로 변환하고 부족수량 합치기
target_pd = all_pd[target_col]
target_np = target_pd.to_numpy()
target_np = target_np.reshape(1,-1)
spare_cnt = max_partcnt - target_np.shape[1]
spare_zero = np.zeros(spare_cnt)
spare_zero = spare_zero.reshape(1,-1)
target_spare_np = np.concatenate([target_np , spare_zero] , axis=1)
print(target_spare_np.shape)

prj_pd

(1, 14) (1, 5976)
(1, 5990) [['20' '20' '1' ... '0' '0' '1']]
(1, 3024)
(1, 9014)
(1, 1000)


,V1,V2,V3,X,Y,NOZZLE_A,NOZZLE_B,NOZZLE_C,SupplyKind_num
0,-1.5665045,-1.2915823,1.8358332,-103.50,19.85,235,1002,240,1
1,-2.5811708,-2.0075417,2.7944536,-88.20,21.65,235,230,0,1
2,-2.5811708,-2.0075417,2.7944536,-86.70,21.65,235,230,0,1
3,-2.0114536,-1.487517,2.132431,-83.70,21.85,235,230,0,1
4,-0.16228016,-0.59209156,1.1140001,-106.70,22.35,230,0,0,1
...,...,...,...,...,...,...,...,...,...
659,-0.13129935,0.80537415,0.85170627,-62.60,177.05,235,0,0,1
660,-0.13129935,0.80537415,0.85170627,-67.30,177.05,235,0,0,1
661,-1.0396583,0.7874358,0.83841246,-117.55,177.25,1002,240,0,1
662,-0.65744805,-0.35548165,0.9733705,-111.15,177.25,240,1002,0,1


In [60]:
#최대 부품 수량 정하기
max_partcnt = 1000

global data_x , data_y

# 부품 데이터 게더링 함수
def part_gathering(path):

    #prj_result 파일 읽어와서 부품의 벡터값 추가
    read_pd = pd.read_csv(path , sep=',' , dtype = 'unicode')
    read_pd[['V1' , 'V2' , 'V3']] = read_pd['NAME_Part'].apply(PartVector_V)

    # 학습에 사용할 컬럼 추출
    all_col = ['V1' , 'V2' , 'V3' , 'X' ,	'Y' , 'NOZZLE_A' , 'NOZZLE_B' , 'NOZZLE_C' , 'SupplyKind_num' , 'HEAD']
    need_col = ['V1' , 'V2' , 'V3' , 'X' ,	'Y' , 'NOZZLE_A' , 'NOZZLE_B' , 'NOZZLE_C' , 'SupplyKind_num']
    target_col = ['HEAD']
    all_pd = read_pd[all_col]

    # Option값 가져와서 넘파이로 변환
    prj_opt = read_pd.loc[1][:14]
    opt_np = prj_opt.to_numpy()
    opt_np = opt_np.reshape(1,-1)
    # print(opt_np)

    # X,Y 좌표 크기로 정렬
    all_pd['X'] = pd.to_numeric(all_pd['X'])
    all_pd['Y'] = pd.to_numeric(all_pd['Y'])
    all_pd = all_pd.sort_values(by = 'X')
    all_pd = all_pd.sort_values(by = 'Y')
    all_pd = all_pd.reset_index(drop=True)
    prj_pd = all_pd[need_col]

    # 부품데이터 넘파이로 변환
    prj_np = prj_pd.to_numpy()
    prj_np = prj_np.reshape(1, -1)
    # print(prj_np)

    # Option과 부품데이터 합치기
    opt_need_np = np.concatenate([opt_np , prj_np] , axis=1)
    # print(opt_np.shape , prj_np.shape )
    # print(opt_need_np.shape , opt_need_np)

    # Max 부품 수량에서 부족수량 채우기
    spare_cnt = max_partcnt * len(need_col) - prj_np.shape[1]
    spare_zero = np.zeros(spare_cnt)
    spare_zero = spare_zero.reshape(1,-1)
    # print(spare_zero.shape)

    # Option + 부품데이터 + 부족수량 합치기
    opt_need_spare_np = np.concatenate([opt_need_np , spare_zero] , axis=1)
    # print(opt_need_spare_np.shape)

    # Taget 데이터 넘파이로 변환하고 부족수량 합치기
    target_pd = all_pd[target_col]
    target_np = target_pd.to_numpy()
    target_np = target_np.reshape(1,-1)
    spare_cnt = max_partcnt - target_np.shape[1]
    spare_zero = np.zeros(spare_cnt)
    spare_zero = spare_zero.reshape(1,-1)
    target_spare_np = np.concatenate([target_np , spare_zero] , axis=1)
    # print(target_spare_np.shape)

    return opt_need_spare_np , target_spare_np

In [63]:
# 파일 리스트 불러와서 넘파이로 데이터셋 구성하기

filelist_re = filelist[:2048]
trainlist = filelist_re[:1632]
vallist = filelist_re[1632:1632+224]
testlist = filelist_re[1632+224:1632+224+192]

data_x , data_y = part_gathering(filelist[0])

for i in trainlist:
    x,y = part_gathering(i)
    data_x = np.concatenate([data_x , x])
    data_y = np.concatenate([data_y , y])
data_x = data_x[1:]
data_y = data_y[1:]
# print(data_x.shape , data_y.shape)

In [64]:
print(data_x.shape , data_y.shape)
print(len(trainlist) , len(vallist) , len(testlist))

(1632, 9014) (1632, 1000)
1632 224 192


In [ ]:
np.save('Head_230927_1_x.npy' , data_x)
np.save('Head_230927_1_y.npy' , data_y)